In [1]:
LOADABLE_ROWS = 1
FILE_NAME = 'ec_instance.txt'

In [2]:
class Matrix:
    def __init__(self, data, rows, cols):
        self.data = data
        self.rows = rows
        self.cols = cols

In [3]:
def count_comment_lines(filename):
    with open(filename, 'r') as f:
        return sum(1 for line in f if line.startswith(";;;"))

In [4]:
def count_total_lines(filename):
    with open(filename, 'r') as f:
        return sum(1 for line in f)

In [5]:
def detect_columns(filename, offset):
    with open(filename, 'r') as f:
        for _ in range(offset):
            next(f)  # skip lines
        line = next(f).strip().split()
        return len(line) - 1  # excluding the trailing "-"

In [6]:
def read_file(filename, rows, cols, offset):
    with open(filename, 'r') as f:
        for _ in range(offset):
            next(f)  # skip lines

        data = []
        for _ in range(rows):
            line = next(f, None)
            if line is None:
                break
            numbers = list(map(int, line.strip().split()[:-1]))  # excluding the trailing "-"
            if len(numbers) != cols:
                break
            data.extend(numbers)

        matrix_rows = len(data) // cols
        return Matrix(data, matrix_rows, cols)

In [7]:
def intersect(row1, row2):
    """Returns True if two rows have a common '1', else False"""
    return any(a == b == 1 for a, b in zip(row1, row2))

def union(row1, row2):
    """Return the union of two rows (bitwise OR)"""
    return [a | b for a, b in zip(row1, row2)]

In [8]:
union([1, 0, 0, 0, 0], [0, 1, 1, 0, 0])

[1, 1, 1, 0, 0]

In [9]:
# def EC(A, B, COV, offset):
#     N = A.rows
#     M = A.cols

#     for i in range(N):
#         # row_data contains the data of the current row
#         # among the N rows of the matrix A
#         # (A is implemented as an array of length N * M)
#         row_data = A.data[i * M:i * M + M] 

#         if sum(row_data) == 0:
#             break

#         if sum(row_data) == M:
#             COV.add((offset + i,))
#             break

#         # Attention: the range of i is [0, N - 1], but the range of j is [0, i + offset]
#         # because A is made of new data only whereas B is increased from the eventual
#         # previous iterations and thus contains more rows than the current A
#         # (if we were to call this algorithm just once with the whole A then
#         # the range of j would be [0, N - 1] but here we might be using a smaller A
#         # of 3 rows whereas B might have 5 rows since we previously worked on the first 2 rows of A)
#         # Since we need the old values of A as well, we are going to read them again
#         # from the file, storing them in a matrix old_A, so that in the following for loop
#         # we are going to use the old values of A (old_A) and the new values of A (A)

# #         for j in range(i + offset):
# #             reading_offset = 0
# #             # j will span from 0 to i+offset excluded and
# #             # when j is less than i, we need to read the old values of A,
# #             # if j is in the first LOADABLE_ROWS rows of A
# #             # then reading_offset will be 0,
# #             # if j is in the second range of LOADABLE_ROWS rows of A
# #             # then reading_offset will be LOADABLE_ROWS,
# #             # so we can compute reading_offset starting from j,
# #             # for example if j is 5 and LOADABLE_ROWS is 3
# #             # then reading_offset will be 3
# #             if j < i:
# #                 # load LOADABLE_ROWS rows from the file
# #                 # starting from reading_offset
# #                 old_A = read_file(FILE_NAME, LOADABLE_ROWS, M, reading_offset)
# #                 reading_offset += LOADABLE_ROWS
# #             # if intersect(A.data[j * M:j * M + M], row_data):
# #                 B[j][i + offset] = 0
# #             else:
# #                 I = {offset + i, j}
# #                 U = union(A.data[j * M:j * M + M], row_data)
# #                 if sum(U) == M:
# #                     COV.add(tuple(sorted(I)))
# #                     B[j][i + offset] = 1
# #                 else:
# #                     B[j][i + offset] = 1
# #                     inter = [k for k in range(j) if B[k][i + offset] and B[k][j]]
# #                     if inter:
# #                         explore(I, U, inter, COV, A, B, offset)

# #     return COV

# def explore(I, U, inter, COV, A, B, offset):
#     N = A.cols
#     for k in inter:
#         i_temp = I.union({k})
#         u_temp = union(U, A.data[k * N:k * N + N])
#         if sum(u_temp) == N:
#             COV.add(tuple(sorted(i_temp)))
#         else:
#             inter_temp = [l for l in inter if l < k and B[l][k]]
#             if inter_temp:
#                 explore(i_temp, u_temp, inter_temp, COV, A, B, offset)



In [10]:
def EC(A, B, COV, offset, FILE_NAME, LOADABLE_ROWS):
    N = A.rows
    M = A.cols

    comment_lines = count_comment_lines(FILE_NAME)

    for i in range(N):
        row_data = A.data[i * M:i * M + M] 

        if sum(row_data) == 0:
            break

        if sum(row_data) == M:
            COV.add((offset + i,))
            break

        # Iterate over previous rows of A in chunks
        reading_offset = 0
        while reading_offset < i + offset:
            old_A = read_file(FILE_NAME, LOADABLE_ROWS, M, reading_offset+comment_lines)
            for j in range(old_A.rows):
                if intersect(old_A.data[j * M:j * M + M], row_data):
                    B[j + reading_offset][i + offset] = 0
                else:
                    I = {offset + i, j + reading_offset}
                    U = union(old_A.data[j * M:j * M + M], row_data)
                    if sum(U) == M:
                        COV.add(tuple(sorted(I)))
                        B[j + reading_offset][i + offset] = 0
                    else:
                        B[j + reading_offset][i + offset] = 1
                        inter = [k for k in range(j + reading_offset) if B[k][i + offset] and B[k][j + reading_offset]]
                        if inter:
                            explore(I, U, inter, COV, B, offset, old_A.cols, FILE_NAME, LOADABLE_ROWS)

            reading_offset += LOADABLE_ROWS

    return COV

# def explore(I, U, inter, COV, A, B, offset):
#     N = A.cols
#     for k in inter:
#         i_temp = I.union({k})
#         u_temp = union(U, A.data[k * N:k * N + N])
#         if sum(u_temp) == N:
#             COV.add(tuple(sorted(i_temp)))
#         else:
#             inter_temp = [l for l in inter if l < k and B[l][k]]
#             if inter_temp:
#                 explore(i_temp, u_temp, inter_temp, COV, A, B, offset)

def explore(I, U, inter, COV, B, offset, M, FILE_NAME, LOADABLE_ROWS):
    N = M
    comment_lines = count_comment_lines(FILE_NAME)
    for k in inter:
        i_temp = I.union({k})
        
        # Get the data corresponding to row 'k' from the file
        chunk_index = (k // LOADABLE_ROWS) * LOADABLE_ROWS
        chunk = read_file(FILE_NAME, LOADABLE_ROWS, M, chunk_index + comment_lines)
        row_data = chunk.data[(k % LOADABLE_ROWS) * M:(k % LOADABLE_ROWS + 1) * M]
        
        u_temp = union(U, row_data)
        if sum(u_temp) == N:
            COV.add(tuple(sorted(i_temp)))
        else:
            inter_temp = [l for l in inter if l < k and B[l][k]]
            if inter_temp:
                explore(i_temp, u_temp, inter_temp, COV, B, offset, M, FILE_NAME, LOADABLE_ROWS)


In [11]:
# Testing how set changes when passed to a function and updated
def test_set(s):
    s.update({1, 2, 3, 6, 7})

s = set({1,2,3,4,5})
test_set(s)
s

{1, 2, 3, 4, 5, 6, 7}

In [12]:
def incremental_process(A, B, COV, offset, FILE_NAME, LOADABLE_ROWS):
    
    # Find exact covers within the new chunk
    chunk_COV = EC(A, B, set(), offset, FILE_NAME, LOADABLE_ROWS)
    print("Chunk COV:", chunk_COV)
    COV.update(chunk_COV)

    return COV

    # # Process intersections of the new chunk with the previous chunks and update B accordingly
    # for i in range(A.rows):
    #     for j in range(N):
    #         if intersect(A.data[i * A.cols: i * A.cols + A.cols], 
    #                      A.data[j * A.cols: j * A.cols + A.cols]):
    #             B[j][offset + i] = 0
    #         else:
    #             B[j][offset + i] = 1

    # # Process intersections within the new chunk and update B accordingly
    # for i in range(A.rows):
    #     for j in range(i+1, A.rows):
    #         if intersect(A.data[i * A.cols: i * A.cols + A.cols],
    #                      A.data[j * A.cols: j * A.cols + A.cols]):
    #             B[offset + i][offset + j] = 0
    #             B[offset + j][offset + i] = 0
    #         else:
    #             B[offset + i][offset + j] = 1
    #             B[offset + j][offset + i] = 1

    #     if sum(A.data[i * A.cols: i * A.cols + A.cols]) == A.cols:
    #         COV.add((offset + i,))


# def main():
#     # FILE_NAME = "ec_instance.txt"

#     mem_size = 1024 * 1024  # 1 MB for now, can adjust as needed

#     comment_lines = count_comment_lines(FILE_NAME)
#     print("Comment lines:", comment_lines)

#     n_columns = detect_columns(FILE_NAME, comment_lines)
#     print(f"Total columns: {n_columns}")


#     # Load the initial chunk
#     # initial_matrix = read_file(FILE_NAME, LOADABLE_ROWS, n_columns, offset)
    
#     total_rows = count_total_lines(FILE_NAME) - comment_lines

#     print(f"Total rows: {total_rows}")
#     B = [[0] * total_rows for _ in range(total_rows)]
#     COV = set()

#     # COV = EC(initial_matrix, B, set(), offset-comment_lines) # offset - comment_lines is 0 at the beginning

#     # offset += initial_matrix.rows
#     # print("Offset after initial chunk:", offset)

#     # print("First B found:")
#     # for row in B:
#     #     print(row)

#     # Process subsequent chunks incrementally

#     offset = 0

#     while True:
#         matrix = read_file(FILE_NAME, LOADABLE_ROWS, n_columns, offset+comment_lines)
#         if not matrix.data:
#             break

#         print("Portion of A read: from row", offset+1, "to row", offset+matrix.rows)
#         # Remember that matrix.data is an array of integers if length N * M
#         # where N is the number of rows and M is the number of columns
#         for i in range(matrix.rows):
#             print(matrix.data[i * matrix.cols:i * matrix.cols + matrix.cols])


#         print("Calling EC")
#         incremental_process(matrix, B, COV, offset, FILE_NAME, LOADABLE_ROWS)

#         print("COV:")
#         print(COV)
        

#         print("B:")
#         for row in B:
#             print(row)

#         offset += matrix.rows

#     print("Solutions:")

#     # Print solutions
#     for solution in COV:
#         sets = [f"S_{i+1}" for i in solution]
#         print(", ".join(sets))

#     print(COV)

# main()

In [34]:
# Creating a function that takes a filename and a number of loadable rows
# and solves the exact cover problem incrementally:

def incremental_exact_cover(filename, loadable_rows):
    comment_lines = count_comment_lines(filename)
    n_columns = detect_columns(filename, comment_lines)
    print(f"Total columns: {n_columns}")
    total_rows = count_total_lines(filename) - comment_lines
    print(f"Total rows: {total_rows}")
    B = [[0] * total_rows for _ in range(total_rows)]
    COV = set()
    offset = 0

    while True:
        matrix = read_file(filename, loadable_rows, n_columns, offset+comment_lines)
        if not matrix.data:
            break

        print("Portion of A read: from row", offset+1, "to row", offset+matrix.rows)
        # for i in range(matrix.rows):
        #     print(matrix.data[i * matrix.cols:i * matrix.cols + matrix.cols])

        incremental_process(matrix, B, COV, offset, filename, loadable_rows)

        print("COV:")
        print(COV)

        offset += matrix.rows

    print("-----------")
    print("COV:")
    print(COV)
    return COV

In [35]:
# Testing the resolution on a small instance
cov = incremental_exact_cover("ec_instance.txt", 2)
print("Solutions:")
for solution in cov:
    sets = [f"S_{i+1}" for i in solution]
    print(", ".join(sets))

Total columns: 10
Total rows: 8
Portion of A read: from row 1 to row 2
Chunk COV: set()
COV:
set()
Portion of A read: from row 3 to row 4
Chunk COV: {(0, 1, 3)}
COV:
{(0, 1, 3)}
Portion of A read: from row 5 to row 6
Chunk COV: {(0, 2, 4)}
COV:
{(0, 1, 3), (0, 2, 4)}
Portion of A read: from row 7 to row 8
Chunk COV: {(1, 5, 6), (4, 5, 7)}
COV:
{(1, 5, 6), (0, 1, 3), (4, 5, 7), (0, 2, 4)}
-----------
COV:
{(1, 5, 6), (0, 1, 3), (4, 5, 7), (0, 2, 4)}
Solutions:
S_2, S_6, S_7
S_1, S_2, S_4
S_5, S_6, S_8
S_1, S_3, S_5


Input sudoku example:

5, 3, 0, 0, 7, 0, 0, 0, 0,
6, 0, 0, 1, 9, 5, 0, 0, 0,
0, 9, 8, 0, 0, 0, 0, 6, 0,
8, 0, 0, 0, 6, 0, 0, 0, 3,
4, 0, 0, 8, 0, 3, 0, 0, 1,
7, 0, 0, 0, 2, 0, 0, 0, 6,
0, 6, 0, 0, 0, 0, 2, 8, 0,
0, 0, 0, 4, 1, 9, 0, 0, 5,
0, 0, 0, 0, 8, 0, 0, 7, 9,

In [36]:
def read_sudoku_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    sudoku = []
    for line in lines:
        row = [int(num) for num in line.strip().split(",") if num]
        sudoku.extend(row)

    return sudoku

# Test
sudoku_sdk_filename = "sudoku2.sdk.txt"
sudoku = read_sudoku_from_file(sudoku_sdk_filename)
for i in range(9):
    print(sudoku[i*9:i*9+9])


[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 0]


In [37]:
def sudoku_to_exact_cover_9_x_9(sudoku):
    N = 9
    constraints = 4  # Cell, Row, Column, Box
    cover_matrix = [[0] * (N * N * constraints) for _ in range(N * N * N)]

    for r in range(N):
        for c in range(N):
            for n in range(1, N + 1):
                # Calculate row index for cover_matrix
                idx = (r * N + c) * N + n - 1
                
                # Cell constraint
                cover_matrix[idx][r * N + c] = 1

                # Row constraint
                cover_matrix[idx][N * N + r * N + n - 1] = 1
                
                # Column constraint
                cover_matrix[idx][2 * N * N + c * N + n - 1] = 1
                
                # Box constraint
                box_row = r // 3
                box_col = c // 3
                box_num = box_row * 3 + box_col
                cover_matrix[idx][3 * N * N + box_num * N + n - 1] = 1
                
    # Prune rows that conflict with given Sudoku puzzle
    rows_to_remove = []
    for r in range(N):
        for c in range(N):
            num = sudoku[r * N + c]
            if num:  # If cell is filled in Sudoku
                start_idx = (r * N + c) * N
                # Remove all rows for this cell, except the one corresponding to 'num'
                for i in range(1, N + 1):
                    if i != num:
                        rows_to_remove.append(start_idx + i - 1)

    # Remove rows in reverse to avoid index issues
    for idx in sorted(rows_to_remove, reverse=True):
        del cover_matrix[idx]
                
    return cover_matrix

In [38]:
def sudoku_to_exact_cover_4_x_4(sudoku):
    N = 4  # This represents the size of the Sudoku (4x4 in this case)
    constraints = 4  # Cell, Row, Column, Box
    cover_matrix = [[0] * (N * N * constraints) for _ in range(N * N * N)]

    for r in range(N):
        for c in range(N):
            for n in range(1, N + 1):
                # Calculate row index for cover_matrix
                idx = (r * N + c) * N + n - 1
                
                # Cell constraint
                cover_matrix[idx][r * N + c] = 1

                # Row constraint
                cover_matrix[idx][N * N + r * N + n - 1] = 1
                
                # Column constraint
                cover_matrix[idx][2 * N * N + c * N + n - 1] = 1
                
                # Box constraint
                box_row = r // 2  # This has changed from 3 to 2 for 4x4 Sudoku
                box_col = c // 2  # This has changed from 3 to 2 for 4x4 Sudoku
                box_num = box_row * 2 + box_col  # This has changed from 3 to 2 for 4x4 Sudoku
                cover_matrix[idx][3 * N * N + box_num * N + n - 1] = 1
                
    # Prune rows that conflict with given Sudoku puzzle
    rows_to_remove = []
    for r in range(N):
        for c in range(N):
            num = sudoku[r * N + c]
            if num:  # If cell is filled in Sudoku
                start_idx = (r * N + c) * N
                # Remove all rows for this cell, except the one corresponding to 'num'
                for i in range(1, N + 1):
                    if i != num:
                        rows_to_remove.append(start_idx + i - 1)

    # Remove rows in reverse to avoid index issues
    for idx in sorted(rows_to_remove, reverse=True):
        del cover_matrix[idx]
                
    return cover_matrix


In [43]:
def exact_cover_solution_to_sudoku_4_x_4(partition):
    N = 4
    solution = [[0] * N for _ in range(N)]
    
    for idx in partition:
        r, c, n = idx // (N * N), (idx // N) % N, (idx % N) + 1
        solution[r][c] = n
        
    return solution

In [39]:
# Function that takes in input a sudoku.sdk filename and in output a sudoku.exc filename
# the default comment is a timestamp
import datetime
def sdk_to_exc(sdk_filename, exc_filename, comment=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), size=9):
    sudoku = read_sudoku_from_file(sdk_filename)
    if size == 4:
        cover_matrix = sudoku_to_exact_cover_4_x_4(sudoku)
    else:
        cover_matrix = sudoku_to_exact_cover_9_x_9(sudoku)
    with open(exc_filename, "w") as f:
        # for each line, append ";;; " at the beginning
        for line in comment.split("\n"):
            f.write(";;; " + line + "\n")
        for row in cover_matrix:
            f.write(" ".join(map(str, row)) + " -\n")

In [40]:
sdk_to_exc("sudoku1.sdk.txt", "sudoku1.exc.txt")
sdk_to_exc("sudoku2.sdk.txt", "sudoku2.exc.txt")

In [41]:
sdk_to_exc("sudoku_easy_4x4.sdk.txt", "sudoku_easy_4x4.exc.txt", size=4)
sdk_to_exc("sudoku_medium_4x4.sdk.txt", "sudoku_medium_4x4.exc.txt", size=4)
sdk_to_exc("sudoku_hard_4x4.sdk.txt", "sudoku_hard_4x4.exc.txt", size=4)


In [42]:
# sol_sudoku_easy_4x4 = incremental_exact_cover("sudoku_easy_4x4.exc.txt", 10)
# print("Solutions:")
# print(sol_sudoku_easy_4x4)

Total columns: 64
Total rows: 46
Portion of A read: from row 1 to row 10
Chunk COV: set()
COV:
set()
Portion of A read: from row 11 to row 20
Chunk COV: set()
COV:
set()
Portion of A read: from row 21 to row 30
Chunk COV: set()
COV:
set()
Portion of A read: from row 31 to row 40
Chunk COV: set()
COV:
set()
Portion of A read: from row 41 to row 46
Chunk COV: {(0, 2, 7, 9, 13, 16, 18, 19, 23, 24, 31, 32, 34, 37, 38, 44)}
COV:
{(0, 2, 7, 9, 13, 16, 18, 19, 23, 24, 31, 32, 34, 37, 38, 44)}
-----------
COV:
{(0, 2, 7, 9, 13, 16, 18, 19, 23, 24, 31, 32, 34, 37, 38, 44)}
Solutions:
{(0, 2, 7, 9, 13, 16, 18, 19, 23, 24, 31, 32, 34, 37, 38, 44)}


In [44]:
# solved_sudoku_easy_4x4 = exact_cover_solution_to_sudoku_4_x_4(sol_sudoku_easy_4x4.pop())
# print("Solved Sudoku:")
# for row in solved_sudoku_easy_4x4:
#     print(row)

Solved Sudoku:
[3, 4, 2, 2]
[4, 4, 1, 4]
[3, 3, 0, 1]
[0, 0, 0, 0]


In [ ]:
# sol_sudoku_medium_4x4 = incremental_exact_cover("sudoku_medium_4x4.exc.txt", 10)
# print("Solutions:")
# print(sol_sudoku_medium_4x4)

In [ ]:
# sol_sudoku_hard_4x4 = incremental_exact_cover("sudoku_hard_4x4.exc.txt", 10)
# print("Solutions:")
# print(sol_sudoku_hard_4x4)

In [18]:
# sol1 = incremental_exact_cover("sudoku1.exc.txt", 50)
# sol1

Portion of A read: from row 1 to row 50
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

KeyboardInterrupt: 

In [28]:
# sol2 = incremental_exact_cover("sudoku2.exc.txt", 5)
# sol2

Portion of A read: from row 1 to row 5
Chunk COV: set()
COV:
set()
Portion of A read: from row 6 to row 10
Chunk COV: set()
COV:
set()
Portion of A read: from row 11 to row 15
Chunk COV: set()
COV:
set()
Portion of A read: from row 16 to row 20


KeyboardInterrupt: 